By AppliedAICourse.com

In [1]:
# Number of CPUs and Threads in Linux.
# Refer: https://linux.die.net/man/1/lscpu
!lscpu


zsh:1: command not found: lscpu


In [2]:
# for Mac
!system_profiler SPHardwareDataType; 

Hardware:

    Hardware Overview:

      Model Name: MacBook Pro
      Model Identifier: MacBookPro16,1
      Processor Name: 6-Core Intel Core i7
      Processor Speed: 2.6 GHz
      Number of Processors: 1
      Total Number of Cores: 6
      L2 Cache (per Core): 256 KB
      L3 Cache: 12 MB
      Hyper-Threading Technology: Enabled
      Memory: 16 GB
      Boot ROM Version: 1554.80.3.0.0 (iBridge: 18.16.14346.0.0,0)
      Serial Number (system): C02ZR5NAMD6M
      Hardware UUID: 2732CCC8-C49F-53A7-ACDF-20160E69F075
      Activation Lock Status: Disabled



## Mean of 100 Million observations

In [37]:
# Generate random 100MM data points 
import numpy as np
n =100000000
d = np.random.rand(n)
print(d.shape)

(100000000,)


In [38]:
import time
def mean():

  #Sum using for loops. We can use inbuilt NumPy Sum opeartion for better speed.
  sum = 0
  n=d.size
  for i in range(n):
    sum +=d[i]

  #Mean
  mean = sum/n
  return mean


#Time the execution
start_time = time.time()
m = mean() # compute mean of 100MM numbers.
end_time = time.time()
print (end_time-start_time)
print(m)

20.49630379676819
0.5000084596707736


### Multi-Processing Code

#### Doesn't run in Jupyter notebook
- Use Script as workaround

In [21]:
#Refer: https://docs.python.org/3/library/multiprocessing.html
from multiprocessing import Process, Queue
import math

def mean_MP(s, e, q, d ):

  #Sum using for loops. We can use inbuilt NumPy Sum opeartion for better speed.
  sum = 0
  for i in range(s,e+1):
    sum +=d[i]

  #Mean
  print(sum)  
  mean = sum/(e-s+1)
  q.put(mean)
  return 

n1 = math.floor(n/2)

q = Queue() #Queues are thread and process safe. For communicating between processes and threads.

p1 = Process(target=mean_MP, args=(0, n1,q,d )) 
p2 = Process(target=mean_MP, args=(n1+1,n-1, q,d)) 

#Time the execution

start_time = time.time()

p1.start()
p2.start()

p1.join() # Wait till p1 finishes
p2.join() 

m=0;
while not q.empty():
     m += q.get()

m /= 2;

end_time = time.time()
print (end_time-start_time)
print(m)

#This doesn't work




KeyboardInterrupt: 

In [40]:
 #using mean_MP function in a script
from multiprocessing import Process, Queue
import math
import mean_MP
import numpy as np

if __name__ == '__main__':
    n1 = math.floor(n/2)

    q = Queue() #Queues are thread and process safe. For communicating between processes and threads.
   
   

    p1 = Process(target=mean_MP.mean_MP, args=(0, n1,q,d )) 
    p2 = Process(target=mean_MP.mean_MP, args=(n1+1,n-1, q,d)) 
    
    #Time the execution
    start_time = time.time()

    p1.start()
    p2.start()

    p1.join() # Wait till p1 finishes
    p2.join() 

    m=0;
    while not q.empty():
         m += q.get()

    m /= 2;

    end_time = time.time()
    print (end_time-start_time)
    print(m)



15.237097978591919
0.5000084596705494


In [44]:
#using mean_MP function in a script
from multiprocessing import Process, Queue
import math
import mean_MP
import numpy as np

if __name__ == '__main__':
    n1 = math.floor(n/2)

    q = Queue() #Queues are thread and process safe. For communicating between processes and threads.
    arg1=0
    arg2=n1
    p_list=[]
    for p in range(2):
        p_list.append(Process(target=mean_MP.mean_MP,args=(arg1,arg2,q,d)))
        arg1=arg2+1
        arg2=n-1
    start_time = time.time()

    p_list[0].start()
    p_list[1].start()

    p_list[0].join() # Wait till p1 finishes
    p_list[1].join() 

    m=0;
    while not q.empty():
         m += q.get()

    m /= 2;

    end_time = time.time()
    print (end_time-start_time)
    print(m)


        

    #p1 = Process(target=mean_MP.mean_MP, args=(0, n1,q,d )) 
    #p2 = Process(target=mean_MP.mean_MP, args=(n1+1,n-1, q,d)) 
    
    #Time the execution

   

    

17.752393007278442
0.5000084596705494


In [45]:
p_list[0]

<Process name='Process-45' pid=44076 parent=8431 stopped exitcode=0>

In [46]:

p_list[1]

<Process name='Process-46' pid=44077 parent=8431 stopped exitcode=0>

0.12081503868103027
0.0


### Multi-Threaded Code

In [39]:
#Refer: https://docs.python.org/3/library/threading.html
from threading import Thread


means = [0,0];

def mean_MT(s, e, threadNum ):

  #Sum using for loops. We can use inbuilt NumPy Sum opeartion for better speed.
  sum = 0
  for i in range(s,e+1):
    sum +=d[i]

  #Mean
  mean = sum/(e-s+1)
  means[threadNum] = mean; # means is a shared varibale between the threads

  return 

n1 = math.floor(n/2)

t1 = Thread(target=mean_MT, args=(0, n1,0 ))  # Third apram is the thread number
t2 = Thread(target=mean_MT, args=(n1+1,n-1,1)) 

#Time the execution
start_time = time.time()

t1.start()
t2.start()

t1.join() # Wait till t1 finishes
t2.join() 

m = (means[0]+means[1])/2
    
end_time = time.time()
print (end_time-start_time)
print(m)



26.87030792236328
0.5000084596705494


## Joblib


### Caching of function output values

In [23]:

#Transparent and fast disk-caching of output value
# Refer: https://joblib.readthedocs.io/en/latest/
from joblib import Memory
cachedir = './'
mem = Memory(cachedir)

import numpy as np
a = np.vander(np.arange(3)).astype(np.float)
square = mem.cache(np.square)
b = square(a) 

________________________________________________________________________________
[Memory] Calling square...
square(array([[0., 0., 1.],
       [1., 1., 1.],
       [4., 2., 1.]]))
___________________________________________________________square - 0.0s, 0.0min


In [24]:
c = square(a)
# The above call did not trigger an evaluation

### Simple Parallel programming for Loops

In [25]:
# Refer: https://joblib.readthedocs.io/en/latest/generated/joblib.Parallel.html

import time
from math import sqrt # inbuilt fucntion

def f(i):
    
    # some computations  that take time
    x=10000
    p =1;
    for j in range(x):
        for k in range(j):
            p *= k
    
    return sqrt(i ** 2);

# Find sqrt of first n numbers
n=10;

start_time = time.time()

for i in range(n):
    f(i)

end_time = time.time()
print (end_time-start_time)

19.42867112159729


In [32]:
from joblib import Parallel, delayed

start_time = time.time()

a = Parallel(n_jobs=2)(delayed(f)(i) for i in range(n)) 

# Why we need dealyed(): https://stackoverflow.com/questions/42220458/what-does-the-delayed-function-do-when-used-with-joblib-in-python

end_time = time.time()
print(a)
print (end_time-start_time)


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
10.564125299453735


In [31]:
a

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [27]:
# Multi threading: GIL is an issue
start_time = time.time()

a = Parallel(n_jobs=2,prefer="threads")(delayed(f)(i ** 2) for i in range(n))

end_time = time.time()
print (end_time-start_time)

19.89794421195984


In [28]:

# 6 jobs

from joblib import Parallel, delayed

start_time = time.time()

a = Parallel(n_jobs=6)(delayed(f)(i ** 2) for i in range(n)) 

# Why we need dealyed(): https://stackoverflow.com/questions/42220458/what-does-the-delayed-function-do-when-used-with-joblib-in-python

end_time = time.time()
print (end_time-start_time)

4.79365611076355
